In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from PIL import Image
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 载入数据集
mnist = input_data.read_data_sets(r"C:\Users\zdwxx\Downloads\Compressed\MNIST_data", one_hot=True)

# 定义每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 初始化权值函数
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) # 生成一个截断的正态分布
    return tf.Variable(initial)

# 初始化偏置值函数
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积层
def conv2d(x, W):
#   x input tensor of shape:`[filter_height * filter_wid3th * in_channels, output_channels]`.
# `[filter_height, filter_width, in_channels, out_channels]`
# `strides[0] = strides[3] = 1,strides[1]->x方向上的补偿，[2]->y方向上的补偿
# padding: A string from:"SAME", "VAILD"
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

# 池化层
def max_pool_2x2(x):
# ksize[1,x,y,1]
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# 改变x的格式转为4D的向量[batch, in_height, in_widch, in_channels]
x_image = tf.reshape(x, [-1, 28, 28, 1]) # [批次，高，宽，通道]

# 初始化第一个卷积层的权值和偏置值
W_conv1 = weight_variable([5, 5, 1, 32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征

b_conv1 = bias_variable([32]) # 每一个卷积核的偏置值



# 把x_image的权值进行卷积，加上偏置值，然后应用于relu激活函数

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # 进行max_pool


# 初始化第二个卷积层的权值和偏置值
W_conv2 = weight_variable([5, 5, 32, 64]) # 5*5的采样窗口，64个卷积核从32个平面抽取特征

b_conv2 = bias_variable([64]) # 每一个卷积核的偏置值


# 把x_image的权值进行卷积，加上偏置值，然后应用于relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) # 进行max_pool

# 28*28的图片第一次卷积后还是28*28，第一次池化后变成了14*14
# 第二次卷积后为14*14，第二次池化后变成了7*7
# 通过上面的操作后得到了64张7*7的平面

# 初始化第一个全连接层的权值
W_fc1 = weight_variable([7*7*64, 1024])

b_fc1 = bias_variable([1024])


# 把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

# 求出第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 使用dropout防止过拟合
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


# 初始化第二个全连接层
W_fc2 = weight_variable([1024, 10])

b_fc2 = bias_variable([10])

# 计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))

# loss = tf.reduce_mean(tf.square(y - prediction))

# Adam优化器
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

# 结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax返回1维张量中最大的值所在的位置
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))#cast转换类型，True->1.0, False->0.0

Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\train-images-idx3-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\t10k-labels-idx1-ubyte.gz


In [4]:
writer = tf.train.Saver()

writer.restore(sess, "C:/model/cnn_mnist/my_mnist_net.ckpt")

INFO:tensorflow:Restoring parameters from C:/model/cnn_mnist/my_mnist_net.ckpt


In [3]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [5]:
sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})

0.9918

In [27]:
img = Image.open("c:/learn/test.jpg")
img = img.convert("L")
x_img_data = (255 - np.array(img)) / 255
x_img_data = x_img_data.reshape((1, 784))
result = sess.run(prediction, feed_dict={x:x_img_data, y:np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]).reshape((1, 10)), keep_prob:1.0})
print("结果是:%d, 可能性是: %.3f%%" % (result.argmax(), result.max() * 100))

结果是:2, 可能性是: 99.983%


In [22]:
img = Image.open("c:/learn/test.jpg")
img = img.convert("L")
x_img_data = (255 - np.array(img)) / 255
x_img_data = x_img_data.reshape((1, 784))
sess.run(train_step, feed_dict={x:x_img_data, y:np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]).reshape((1, 10)), keep_prob:1.0})

In [26]:
result.argsort()

array([[4, 6, 9, 1, 5, 8, 0, 3, 7, 2]], dtype=int64)